# Basket QRBS with Noise

This notebook introduces how to use noisy QPUs for evaluating the basket problem. 

**BE AWARE** This notebook can only be used inside a **EVIDEN QLM**

### 0. Initial imports and configuration

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../")
import itertools as it
import pandas as pd
import numpy as np
from qat.core import Job, Batch

In [ ]:
# starting SelectableQPU object
from selectable_qpu import SelectableQPU
qpu_selected = SelectableQPU()

In [ ]:
# For ideal qpus selection
from qpu.select_qpu import select_qpu
from qpu.benchmark_utils import combination_for_list
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config_c = {
    "qpu_type": ideal_qpus[2], 
}
ideal_qpu = select_qpu(qpu_config_c)

In [ ]:
# The basket QRBS
from basket import basquet_qrbs

The players used for evaluation

In [ ]:
Name = ["Elias", "Blas", "Luis", "Juan", "Raul", "Cholo"]
Throws = [16, 17, 17, 15, 18, 18]
Heights = [198, 193, 188, 203, 176, 186]

## 1. Ideal Solution

First thing we are going to used ideal simulation using the different models of the QRBS for comparison purpouses

In [ ]:
final_model_ideal = []
for model in ["cf", "fuzzy", "bayes"]:

    player_evaluation_ideal = [
        basquet_qrbs(t, h, qpu_selected, type_qpu=ideal_qpu, shots=0, model=model) for t, h in zip(Throws, Heights)
    ]
    pdf_ideal = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_ideal)]
    pdf_ideal = pd.DataFrame(
        pdf_ideal,
        columns = ["Name", "Throws", "Height", "Final_Score"]
    )
    pdf_ideal.sort_values(["Final_Score"], ascending=False, inplace=True)
    final_model_ideal.append(pdf_ideal)

The 3 models (**CF**, **FUZZY** y **BAYES**) should output the same scores

In [ ]:
#cf
final_model_ideal[0]

In [ ]:
# fuzzy
final_model_ideal[1]

In [ ]:
# bayes
final_model_ideal[2]

In [ ]:
# cf vs fuzzy
np.isclose(final_model_ideal[0]["Final_Score"], final_model_ideal[1]["Final_Score"] )

In [ ]:
# cf vs bayes
np.isclose(final_model_ideal[0]["Final_Score"], final_model_ideal[2]["Final_Score"] )

In [ ]:
# ideal results
final_model_ideal[0]

## 2. Circuit Rewriting

Now we are going to simulate the system using an ideal QPU but we are going to rewrite the quantum circuits generated by the basket **QRBS**. We need to instantiate another **QPU**. We are going to use the following dictionary where we have updated the value of the **qpu_type** key to **ideal** and the **kak_compiler** key to *ZXZ* (consecutive one qubit gates will be merged in one unitary operator and decompose following the pattern: $R_z-R_x-R_z$). In this case, by default, the **LinaAlg** algebra simulator will be used for doing the computations.


In [ ]:
rewriter_qpu = [{
    "qpu_type": ["ideal"],
    "kak_compiler" : ["ZXZ"],
    "qpu_name" : ["ideal"],
    "t_gate_1qb" : [None],
    "t_gate_2qbs" : [None],
    "t_readout": [None],
    "sim_method" : [
        {
        "sim_method": "deterministic",
        "n_samples" : None,
        "bond_dimension": None
        },
        {
        "sim_method": "stochastic",
        "n_samples" : 100, 
        "bond_dimension": None            
        },
        {
        "sim_method": "mpo",
        "n_samples" : None,
        "bond_dimension": 16
        },        
    ],
    "n_samples" : [1000],
    "depol_channel" : [{
        "active": False,
        "error_gate_1qb" : None,
        "error_gate_2qbs" : None
    }],
    "idle" : [{
        "amplitude_damping": False,
        "dephasing_channel": False,
        "t1" : None,
        "t2" : None
    }],
    "meas": [{
        "active":False,
        "readout_error": None
    }]
}]
ideal_rewriter = combination_for_list(rewriter_qpu)
ideal_rewriter_qpu = select_qpu(ideal_rewriter[0])

To show how the circuit rewritter works we are going to extract one ideal quantum circuit from the QRBS and we are going to compile it using the rewritter.

In [ ]:
# Get the qrb basket
qrb_basket = basquet_qrbs(18, 196, qpu_selected, type_qpu=ideal_qpu, shots=0, model="cf")["qrbs"]
# Get and island
island = qrb_basket._engine._islands[0]
# select builder
builder = qpu_selected.BUILDERS["cf"]
circuit = island.build(builder)
# This is one ideal quantum circuit used by the basket qrbs
%qatdisplay circuit

In [ ]:
# Now we compile the circuit using the Rewrittter circuit plugins: 
new_batch = ideal_rewriter_qpu.compile(Batch(jobs=[circuit.to_job()]))
new_circ = new_batch.jobs[0].circuit
%qatdisplay new_circ

As can be seen the $M$ gates from the **QRBS** circuit were decomposed following the desired pattern ($R_z-R_x-R_z$ in the example), and all the *Toffoli* gates were transformed to combinations of 2 and 1 qubit gates.

In this case, the **ideal_rewriter_qpu** only rewrittes the circuit but uses ideal simulation so the results should be the same than in the case without rewritting

In [ ]:
final_model_rewriter = []
for model in ["cf", "fuzzy", "bayes"]:
    player_evaluation_ideal_rewriter = [
        basquet_qrbs(t, h, qpu_selected, type_qpu=ideal_rewriter_qpu, shots=0, model=model) for t, h in zip(Throws, Heights)
    ]
    pdf_ideal_rewriter = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_ideal_rewriter)]
    pdf_ideal_rewriter = pd.DataFrame(
        pdf_ideal_rewriter,
        columns = ["Name", "Throws", "Height", "Final_Score"]
    )
    pdf_ideal_rewriter.sort_values(["Final_Score"], ascending=False, inplace=True)
    
    final_model_rewriter.append(pdf_ideal_rewriter)

In this case all the simulations should be equal to the initial ideal ones!!

In [ ]:
# Each simulation with circuit rewritting is compared without circuit rewritting
for i in range(3):
    print(np.isclose(final_model_ideal[i]["Final_Score"], final_model_rewriter[i]["Final_Score"] ))

## 3. Noisy Simulations

The **qpu** package allows the user to cofigure noisy simulations in an easy way (by providing a Python configuration dictionary). Here we show how to use it.

We are going to load the noisy configurations from a JSON files previously generated,

In [ ]:
import json

In [ ]:
with open("../qpu/qpu_noisy_deterministic.json") as json_file:
    noisy_cfg = json.load(json_file)
qpu_list = combination_for_list(noisy_cfg)

The JSON file is loaded as a list of Python dictionaries where each element configures different noisy models.

In [ ]:
len(qpu_list)

In [ ]:
qpu_list[1]

In [ ]:
qpu_list[-1]

In [ ]:
# select a noisy model configuration
noisy_id = -1
noisy_qpu_cfg = qpu_list[noisy_id]
print(noisy_qpu_cfg)

Once the dictionary configuration is selected it should be provided to the *select_qpu* and the properly configured noisy **QPU** object will be returned.

In [ ]:
noisy_qpu = select_qpu(noisy_qpu_cfg)

In [ ]:
print(noisy_qpu)
# Error of 1-qubit gates
print("Error rate for: 1-qubit gates: {}".format(
    noisy_qpu.hardware_model.gate_noise["H"].keywords["rb_eps"])
)
#  == noisy_qpu_cfg["depol_channel"]["error_gate_1qb"])
# Error of 2-qubits gates
print("Error rate for: 2-qubits gates: {}".format(
    noisy_qpu.hardware_model.gate_noise["CNOT"].keywords["rb_eps"])
)
# T1 for Amplitude Damping
print("T1 time: {}".format(
    noisy_qpu.hardware_model.idle_noise[0].T_1)
     )
# Tvarphi for Dephaising
print("T1 time: {}".format(
    noisy_qpu.hardware_model.idle_noise[0].T_1)
     )

print("Tvarphi time: {}".format(noisy_qpu.hardware_model.idle_noise[1].T_phi))


Now we can provide the new **noisy_qpu** to the *basquet_qrbs*. 

**BE AWARE**
*deterministic* simulation (*deterministic* in **sim_method** key) will fail for **CF** and **FUZZY** models becuase they generated big circuits (no more than 15 qubits circuits can be simulated with this method). In this case, stochastic simulation shoul be used (the user can load the *../qpu/qpu_noisy_stochastic.json* JSON file where several nosiy models for this kind of simulation were configured)

In [ ]:
model = "bayes"
shots = 0
player_evaluation_noisy = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=noisy_qpu, shots=shots, model=model) for t, h in zip(Throws, Heights)
]

In [ ]:
pdf_noisy = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_noisy)]
pdf_noisy = pd.DataFrame(
    pdf_noisy,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf_noisy.sort_values(["Final_Score"], ascending=False, inplace=True)
pdf_noisy.reset_index(drop=True, inplace=True)


In [ ]:
pdf_noisy

In [ ]:
pdf_ideal

## 5. launch_basket.py script

Additionally a script called *launch_basket.py* was developed. This script allows to execute the basket qrb from command line. 

Use the following command for getting the help:

    python launch_basket.py -h
    
The help menu is the following:

    Module Not Found
    usage: launch_basket.py [-h] [-players PLAYERS] [-json_qpu JSON_QPU] [-model MODEL] [-id ID]
                            [-shots SHOTS] [-name BASE_NAME] [--count] [--all] [--print] [--save] [--test]
                            [-folder FOLDER_PATH] [--exe]

    optional arguments:
      -h, --help           show this help message and exit
      -players PLAYERS     csv with the player list to evaluate
      -json_qpu JSON_QPU   JSON with the qpu configuration
      -model MODEL         cf, bayes, fuzzy
      -id ID               For executing only one element of the list
      -shots SHOTS         For executing only one element of the list
      -name BASE_NAME      Additional name for the generated files
      --count              For counting elements on the list
      --all                For executing complete list
      --print              For printing the AE algorihtm configuration.
      --save               For saving staff
      --test               Only one player will be evaluated.
      -folder FOLDER_PATH  Path for storing folder
      --exe                For executing program
      
      
The players for evaluating should be given as a CSV file (an example of the file is provided in the **basket_players.csv**). 

An example of execution will be:

    python launch_basket.py -players ./basket_players.csv -json_qpu qpu/qpu_ideal.json -model cf -shots 0 -id 2 --print --exe --save -name players -folder ./results/
    
In this case the *qpu/qpu_ideal.json* file is used for configuring the QPU (the selected qpu will be the second one: -id 2). The model for the QRBS will be certainty factors (-model cf) simulation will be ideal ( -shots 0). The results will be saved (--save) in the folder ./results (-folder ./results/) and the obtainted file will have as base name players (-name players.


Noisy simulations can be executed by providing a JSON file properly configured (like for example **../qpu/qpu_noisy_deterministic.json** or **../qpu/qpu_noisy_stochastic.json**)